In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = (
    SparkSession.builder
        .appName("MongoToBronze")
        .config("spark.jars.packages",
                "org.mongodb.spark:mongo-spark-connector_2.12:10.3.0,"
                "io.delta:delta-spark_2.12:3.2.1")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("spark.mongodb.read.connection.uri",
                "mongodb://host.docker.internal:27017/mongo_movies.movies")  
        .getOrCreate()
)
df = spark.read.format("mongodb").load()



df_clean = (
    df.withColumnRenamed("Revenue (Millions)", "Revenue_Millions")
      .withColumnRenamed("Runtime (Minutes)", "Runtime_Minutes")
      .drop("views")
)
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")
spark.conf.set("delta.columnMapping.mode", "name")

bronze_path = "hdfs://master:8020/delta/bronze/movies"
df_clean.write.format("delta") \
    .option("delta.columnMapping.mode", "name") \
    .mode("overwrite") \
    .save(bronze_path)


print(f" Bronze Layer saved to {bronze_path}")

[Stage 1:>                                                          (0 + 1) / 1]

 Bronze Layer saved to hdfs://master:8020/delta/bronze/movies
